Submission Login 	gimin@outlook.com

Submission Password 	7QuJFUEHAy

In [13]:
from vecutil import list2vec
from GF2 import one
from solver import solve
from matutil import listlist2mat, coldict2mat, mat2coldict, mat2rowdict
from mat import Mat
from vec import Vec
from The_Basis_problems import vec2rep, exchange, is_independent, is_superfluous
from independence import rank
from triangular import triangular_solve

vecrep(veclist, v)

        Input: veclist, a list of Vecs; v a Vec in the Span of set(veclist)
        
        Output: Vec(u), whose domain is set(range(len(veclist))) that is the coordinate reprsentation of v, 
                with respect to veclist
       
exchange(S,A,z)

    Input: S set of Vecs (not necessary linearly independent); A subset of S; z instance of Vec such that 
            A.union({z}) linearly independent
    
    Output: Vec(w) is S-A: Span(S) = Span(S.union({z})-{w})

In [2]:
## 3: (Problem 6.7.4) Morph Lemma Coding
def morph(S, B):
    '''
    Input:
        - S: a list of distinct Vecs
        - B: a list of linearly independent Vecs all in Span S
    Output: a list of pairs of vectors to inject and eject (see problem description)
    Example:
        >>> # This is how our morph works.  Yours may yield different results.
        >>> # Note: Make a copy of S to modify instead of modifying S itself.
        >>> from vecutil import list2vec
        >>> from vec import Vec
        >>> S = [list2vec(v) for v in [[1,0,0],[0,1,0],[0,0,1]]]
        >>> B = [list2vec(v) for v in [[1,1,0],[0,1,1],[1,0,1]]]
        >>> D = {0, 1, 2}
        >>> morph(S, B) == [(Vec(D,{0: 1, 1: 1, 2: 0}), Vec(D,{0: 1, 1: 0, 2: 0})), (Vec(D,{0: 0, 1: 1, 2: 1}), Vec(D,{0: 0, 1: 1, 2: 0})), (Vec(D,{0: 1, 1: 0, 2: 1}), Vec(D,{0: 0, 1: 0, 2: 1}))]
        True
        >>> S == [list2vec(v) for v in [[1,0,0],[0,1,0],[0,0,1]]]
        True
        >>> B == [list2vec(v) for v in [[1,1,0],[0,1,1],[1,0,1]]]
        True
        >>> from GF2 import one
        >>> D = {0, 1, 2, 3, 4, 5, 6, 7}
        >>> S = [Vec(D,{1: one, 2: one, 3: one, 4: one}), Vec(D,{1: one, 3: one}), Vec(D,{0: one, 1: one, 3: one, 5: one, 6: one}), Vec(D,{3: one, 4: one}), Vec(D,{3: one, 5: one, 6: one})]
        >>> B = [Vec(D,{2: one, 4: one}), Vec(D,{0: one, 1: one, 2: one, 3: one, 4: one, 5: one, 6: one}), Vec(D,{0: one, 1: one, 2: one, 5: one, 6: one})]
        >>> sol = morph(S, B)
        >>> sol == [(B[0],S[0]), (B[1],S[2]), (B[2],S[3])] or sol == [(B[0],S[1]), (B[1],S[2]), (B[2],S[3])]
        True
        >>> # Should work the same regardless of order of S
        >>> from random import random
        >>> sol = morph(sorted(S, key=lambda x:random()), B)
        >>> sol == [(B[0],S[0]), (B[1],S[2]), (B[2],S[3])] or sol == [(B[0],S[1]), (B[1],S[2]), (B[2],S[3])]
        True
    '''
    S0 = set(S)  # follows the Morphing Lemma proof algorithm
    A = set()
    L = [] # the list to be returned
    for z in B:
        A.add(z)
        w = exchange(S0, A, z)
        S0 = (S0.union({z})).difference({w})
        L.append((z, w))
    return L
   

In [3]:
S = [list2vec(v) for v in [[1,0,0],[0,1,0],[0,0,1]]]
B = [list2vec(v) for v in [[1,1,0],[0,1,1],[1,0,1]]]
D = {0, 1, 2}
morph(S, B) == [(Vec(D,{0: 1, 1: 1, 2: 0}), Vec(D,{0: 1, 1: 0, 2: 0})), (Vec(D,{0: 0, 1: 1, 2: 1}), Vec(D,{0: 0, 1: 1, 2: 0})), (Vec(D,{0: 1, 1: 0, 2: 1}), Vec(D,{0: 0, 1: 0, 2: 1}))]

True

In [4]:
S == [list2vec(v) for v in [[1,0,0],[0,1,0],[0,0,1]]]

True

In [5]:
B == [list2vec(v) for v in [[1,1,0],[0,1,1],[1,0,1]]]

True

In [6]:
D = {0, 1, 2, 3, 4, 5, 6, 7}
S = [Vec(D,{1: one, 2: one, 3: one, 4: one}), Vec(D,{1: one, 3: one}), Vec(D,{0: one, 1: one, 3: one, 5: one, 6: one}), Vec(D,{3: one, 4: one}), Vec(D,{3: one, 5: one, 6: one})]
B = [Vec(D,{2: one, 4: one}), Vec(D,{0: one, 1: one, 2: one, 3: one, 4: one, 5: one, 6: one}), Vec(D,{0: one, 1: one, 2: one, 5: one, 6: one})]
sol = morph(S, B)
sol == [(B[0],S[0]), (B[1],S[2]), (B[2],S[3])] or sol == [(B[0],S[1]), (B[1],S[2]), (B[2],S[3])]

True

In [7]:
from random import random
sol = morph(sorted(S, key=lambda x:random()), B)
sol == [(B[0],S[0]), (B[1],S[2]), (B[2],S[3])] or sol == [(B[0],S[1]), (B[1],S[2]), (B[2],S[3])]

True

In [8]:
S = [list2vec(v) for v in [[2,4,0],[1,0,3],[0,4,4],[1,1,1]]]
B = [list2vec(v) for v in [[1,0,0],[0,1,0],[0,0,1]]]
for (z,w) in morph(S, B):
    print("injecting ", z)
    print("ejecting ", w)
    print()


injecting  
 0 1 2
------
 1 0 0
ejecting  
 0 1 2
------
 2 4 0

injecting  
 0 1 2
------
 0 1 0
ejecting  
 0 1 2
------
 1 1 1

injecting  
 0 1 2
------
 0 0 1
ejecting  
 0 1 2
------
 1 0 3



In [3]:
## 5: () Subset Basis
def subset_basis(T):
    '''
    Input:
        - T: a set of Vecs
    Output: 
        - set S containing Vecs from T that is a basis for Span T.
    Examples:
        The following tests use the procedure is_independent, provided in module independence
        
        >>> from vec import Vec
        >>> from independence import is_independent
        >>> a0 = Vec({'a','b','c','d'}, {'a':1})
        >>> a1 = Vec({'a','b','c','d'}, {'b':1})
        >>> a2 = Vec({'a','b','c','d'}, {'c':1})
        >>> a3 = Vec({'a','b','c','d'}, {'a':1,'c':3})
        >>> sb = subset_basis({a0, a1, a2, a3})
        >>> len(sb)
        3
        >>> all(v in [a0, a1, a2, a3] for v in sb)
        True
        >>> is_independent(sb)
        True

        >>> b0 = Vec({0,1,2,3},{0:2,1:2,3:4})
        >>> b1 = Vec({0,1,2,3},{0:1,1:1})
        >>> b2 = Vec({0,1,2,3},{2:3,3:4})
        >>> b3 = Vec({0,1,2,3},{3:3})
        >>> sb = subset_basis({b0, b1, b2, b3})
        >>> len(sb)
        3
        >>> all(v in [b0, b1, b2, b3] for v in sb)
        True
        >>> is_independent(sb)
        True

        >>> D = {'a','b','c','d'}
        >>> c0, c1, c2, c3, c4 = Vec(D,{'d': one, 'c': one}), Vec(D,{'d': one, 'a': one, 'c': one, 'b': one}), Vec(D,{'a': one}), Vec(D,{}), Vec(D,{'d': one, 'a': one, 'b': one})
        >>> subset_basis({c0,c1,c2,c3,c4}) == {c0,c1,c2,c4}
        True
    '''
   
    S = set()
   
    for v in T:
        S0 = S.union({v})
        if is_independent(S0): S.add(v)
    return S
    

In [87]:
from vec import Vec
from independence import is_independent
a0 = Vec({'a','b','c','d'}, {'a':1})
a1 = Vec({'a','b','c','d'}, {'b':1})
a2 = Vec({'a','b','c','d'}, {'c':1})
a3 = Vec({'a','b','c','d'}, {'a':1,'c':3})
sb = subset_basis({a0, a1, a2, a3})
len(sb)

3

In [88]:
all(v in [a0, a1, a2, a3] for v in sb)

True

In [89]:
is_independent(sb)

True

In [90]:
D = {'a','b','c','d'}
c0, c1, c2, c3, c4 = Vec(D,{'d': one, 'c': one}), Vec(D,{'d': one, 'a': one, 'c': one, 'b': one}), Vec(D,{'a': one}), Vec(D,{}), Vec(D,{'d': one, 'a': one, 'b': one})
subset_basis({c0,c1,c2,c3,c4}) == {c0,c1,c2,c4}

True

In [4]:
## 6: () Superset Basis Lemma in Python
def superset_basis(C, T):
    '''
    Input:
        - C: linearly independent set of Vecs
        - T: set of Vecs such that every Vec in C is in Span(T)
    Output:
        Linearly independent set S consisting of all Vecs in C and some in T
        such that the span of S is the span of T (i.e. S is a basis for the span
        of T).
    Example:
        >>> from vec import Vec
        >>> from independence import is_independent
        >>> a0 = Vec({'a','b','c','d'}, {'a':1})
        >>> a1 = Vec({'a','b','c','d'}, {'b':1})
        >>> a2 = Vec({'a','b','c','d'}, {'c':1})
        >>> a3 = Vec({'a','b','c','d'}, {'a':1,'c':3})
        >>> sb = superset_basis({a0, a3}, {a0, a1, a2})
        >>> a0 in sb and a3 in sb
        True
        >>> is_independent(sb)
        True
        >>> all(x in [a0,a1,a2,a3] for x in sb)
        True
    '''
    S = C
    
    for v in T.difference(C):
        S0 = S.union({v})
        if is_independent(S0): S.add(v)
    return S

In [107]:
from independence import is_independent
a0 = Vec({'a','b','c','d'}, {'a':1})
a1 = Vec({'a','b','c','d'}, {'b':1})
a2 = Vec({'a','b','c','d'}, {'c':1})
a3 = Vec({'a','b','c','d'}, {'a':1,'c':3})
sb = superset_basis({a0, a3}, {a0, a1, a2})
a0 in sb and a3 in sb

True

In [108]:
is_independent(sb)

True

In [109]:
 all(x in [a0,a1,a2,a3] for x in sb)

True

In [5]:
## 7: (Problem 6.7.6) My Is Independent Procedure
def my_is_independent(L):
    '''
    Input:
        - L: a list of Vecs
    Output:
        - boolean: true if the list is linearly independent
    Examples:
        >>> D = {0, 1, 2}
        >>> L = [Vec(D,{0: 1}), Vec(D,{1: 1}), Vec(D,{2: 1}), Vec(D,{0: 1, 1: 1, 2: 1}), Vec(D,{0: 1, 1: 1}), Vec(D,{1: 1, 2: 1})]
        >>> my_is_independent(L)
        False
        >>> my_is_independent(L[:2])
        True
        >>> my_is_independent(L[:3])
        True
        >>> my_is_independent(L[1:4])
        True
        >>> my_is_independent(L[0:4])
        False
        >>> my_is_independent(L[2:])
        False
        >>> my_is_independent(L[2:5])
        False
        >>> L == [Vec(D,{0: 1}), Vec(D,{1: 1}), Vec(D,{2: 1}), Vec(D,{0: 1, 1: 1, 2: 1}), Vec(D,{0: 1, 1: 1}), Vec(D,{1: 1, 2: 1})]
        True
    '''
    return True if rank(L) == len(L) else False

In [113]:
D = {0, 1, 2}
L = [Vec(D,{0: 1}), Vec(D,{1: 1}), Vec(D,{2: 1}), Vec(D,{0: 1, 1: 1, 2: 1}), Vec(D,{0: 1, 1: 1}), Vec(D,{1: 1, 2: 1})]
my_is_independent(L)

False

In [114]:
my_is_independent(L[:2])

True

In [115]:
my_is_independent(L[:3])

True

In [116]:
my_is_independent(L[1:4])

True

In [117]:
my_is_independent(L[0:4])

False

In [118]:
my_is_independent(L[2:])

False

In [119]:
my_is_independent(L[2:5])

False

In [120]:
L == [Vec(D,{0: 1}), Vec(D,{1: 1}), Vec(D,{2: 1}), Vec(D,{0: 1, 1: 1, 2: 1}), Vec(D,{0: 1, 1: 1}), Vec(D,{1: 1, 2: 1})]

True

In [6]:
## 8: (Problem 6.7.7) My Rank
def my_rank(L):
    '''
    Input: 
        - L: a list of Vecs
    Output: 
        - the rank of the list of Vecs
    Example:
        >>> L = [list2vec(v) for v in [[1,2,3],[4,5,6],[1.1,1.1,1.1]]]
        >>> my_rank(L)
        2
        >>> L == [list2vec(v) for v in [[1,2,3],[4,5,6],[1.1,1.1,1.1]]]
        True
        >>> my_rank([list2vec(v) for v in [[1,1,1],[2,2,2],[3,3,3],[4,4,4],[123,432,123]]])
        2
    '''
    return len(subset_basis(set(L)))

In [7]:
L = [list2vec(v) for v in [[1,2,3],[4,5,6],[1.1,1.1,1.1]]]
my_rank(L)

2

In [8]:
L == [list2vec(v) for v in [[1,2,3],[4,5,6],[1.1,1.1,1.1]]]

True

In [9]:
my_rank([list2vec(v) for v in [[1,1,1],[2,2,2],[3,3,3],[4,4,4],[123,432,123]]])

2

In [16]:
# Ungraded Problem
# use subset_basis to get the basis of the Span. If the bases are linearly independent then u intersection v = 0 
U = {Vec({0, 1, 2, 3}, {0:one, 2:one}), Vec({0, 1, 2, 3}, {2:one})}
V = {Vec({0, 1, 2, 3}, {1:one, 3:one}), Vec({0, 1, 2, 3}, {3:one})}

U1 = {Vec({0, 1, 2}, {0:1, 1:2, 2:3}), Vec({0, 1, 2}, {0:1, 1:2})}
V1 = {Vec({0, 1, 2}, {0:2, 1:1, 2:3}), Vec({0, 1, 2}, {0:2, 1:1, 2:3})}

U2 = {Vec({0, 1, 2, 3}, {0:2, 2:8}), Vec({0, 1, 2, 3}, {0:1, 1:1, 2:4})}
V2 = {Vec({0, 1, 2, 3}, {0:2, 1:1, 2:1, 3:1}), Vec({0, 1, 2, 3},{1:1, 2:1, 3:1})} 


In [19]:
basis_span_U = subset_basis(U)
basis_span_V = subset_basis(V)

basis_span_U1 = subset_basis(U1)
basis_span_V1 = subset_basis(V1)

basis_span_U2 = subset_basis(U2)
basis_span_V2 = subset_basis(V2)

In [20]:
my_is_independent(list(basis_span_U)+list(basis_span_V))

True

In [21]:
my_is_independent(list(basis_span_U1)+list(basis_span_V1))

True

In [22]:
my_is_independent(list(basis_span_U2)+list(basis_span_V2))

True

In [7]:
## 9: (Problem 6.7.11) Direct Sum Unique Representation
def direct_sum_decompose(U_basis, V_basis, w):
    '''
    Input:
        - U_basis: a list of Vecs forming a basis for a vector space U
        - V_basis: a list of Vecs forming a basis for a vector space V
        - w: a Vec in the direct sum of U and V
    Output:
        - a pair (u, v) such that u + v = w, u is in U, v is in V
    Example:

        >>> D = {0,1,2,3,4,5}
        >>> U_basis = [Vec(D,{0: 2, 1: 1, 2: 0, 3: 0, 4: 6, 5: 0}), Vec(D,{0: 11, 1: 5, 2: 0, 3: 0, 4: 1, 5: 0}), Vec(D,{0: 3, 1: 1.5, 2: 0, 3: 0, 4: 7.5, 5: 0})]
        >>> V_basis = [Vec(D,{0: 0, 1: 0, 2: 7, 3: 0, 4: 0, 5: 1}), Vec(D,{0: 0, 1: 0, 2: 15, 3: 0, 4: 0, 5: 2})]
        >>> w = Vec(D,{0: 2, 1: 5, 2: 0, 3: 0, 4: 1, 5: 0})
        >>> (u, v) = direct_sum_decompose(U_basis, V_basis, w)
        >>> (u + v - w).is_almost_zero()
        True
        >>> U_matrix = coldict2mat(U_basis)
        >>> V_matrix = coldict2mat(V_basis)
        >>> (u - U_matrix*solve(U_matrix, u)).is_almost_zero()
        True
        >>> (v - V_matrix*solve(V_matrix, v)).is_almost_zero()
        True
        >>> ww = Vec(D,{0: 2, 1: 5, 2: 51, 4: 1, 5: 7})
        >>> (u, v) = direct_sum_decompose(U_basis, V_basis, ww)
        >>> (u + v - ww).is_almost_zero()
        True
        >>> (u - U_matrix*solve(U_matrix, u)).is_almost_zero()
        True
        >>> (v - V_matrix*solve(V_matrix, v)).is_almost_zero()
        True
        >>> U_basis == [Vec(D,{0: 2, 1: 1, 2: 0, 3: 0, 4: 6, 5: 0}), Vec(D,{0: 11, 1: 5, 2: 0, 3: 0, 4: 1, 5: 0}), Vec(D,{0: 3, 1: 1.5, 2: 0, 3: 0, 4: 7.5, 5: 0})]
        True
        >>> V_basis == [Vec(D,{0: 0, 1: 0, 2: 7, 3: 0, 4: 0, 5: 1}), Vec(D,{0: 0, 1: 0, 2: 15, 3: 0, 4: 0, 5: 2})]
        True
        >>> w == Vec(D,{0: 2, 1: 5, 2: 0, 3: 0, 4: 1, 5: 0})
        True
    '''
    direct_sum_basis = U_basis+V_basis
    x = vec2rep(direct_sum_basis, w) # len(x) = len(direct_sum_basis)
    u = Vec(U_basis[0].D, {}); v = Vec(V_basis[0].D, {})
    
    for i in range(len(U_basis)):
        u = u+x[i]*U_basis[i]
    for j in range(len(V_basis)):
        v = v+x[len(U_basis)+j]*V_basis[j]
    
    return (u, v)

In [116]:
D = {0,1,2,3,4,5}
U_basis = [Vec(D,{0: 2, 1: 1, 2: 0, 3: 0, 4: 6, 5: 0}), Vec(D,{0: 11, 1: 5, 2: 0, 3: 0, 4: 1, 5: 0}), Vec(D,{0: 3, 1: 1.5, 2: 0, 3: 0, 4: 7.5, 5: 0})]
V_basis = [Vec(D,{0: 0, 1: 0, 2: 7, 3: 0, 4: 0, 5: 1}), Vec(D,{0: 0, 1: 0, 2: 15, 3: 0, 4: 0, 5: 2})]
w = Vec(D,{0: 2, 1: 5, 2: 0, 3: 0, 4: 1, 5: 0})
(u, v) = direct_sum_decompose(U_basis, V_basis, w)
(u + v - w).is_almost_zero()

True

In [117]:
U_matrix = coldict2mat(U_basis)
V_matrix = coldict2mat(V_basis)
(u - U_matrix*solve(U_matrix, u)).is_almost_zero()

True

In [118]:
(v - V_matrix*solve(V_matrix, v)).is_almost_zero()

True

In [119]:
ww = Vec(D,{0: 2, 1: 5, 2: 51, 4: 1, 5: 7})
(u, v) = direct_sum_decompose(U_basis, V_basis, ww)
(u + v - ww).is_almost_zero()

True

In [120]:
(u - U_matrix*solve(U_matrix, u)).is_almost_zero()

True

In [121]:
(v - V_matrix*solve(V_matrix, v)).is_almost_zero()

True

In [122]:
U_basis == [Vec(D,{0: 2, 1: 1, 2: 0, 3: 0, 4: 6, 5: 0}), Vec(D,{0: 11, 1: 5, 2: 0, 3: 0, 4: 1, 5: 0}), Vec(D,{0: 3, 1: 1.5, 2: 0, 3: 0, 4: 7.5, 5: 0})]

True

In [123]:
V_basis == [Vec(D,{0: 0, 1: 0, 2: 7, 3: 0, 4: 0, 5: 1}), Vec(D,{0: 0, 1: 0, 2: 15, 3: 0, 4: 0, 5: 2})]

True

In [124]:
w == Vec(D,{0: 2, 1: 5, 2: 0, 3: 0, 4: 1, 5: 0})

True

In [8]:
## 10: (Problem 6.7.12) Is Invertible Function
def is_invertible(M):
    '''
    input: A matrix, M
    outpit: A boolean indicating if M is invertible.

    >>> M = Mat(({0, 1, 2, 3}, {0, 1, 2, 3}), {(0, 1): 0, (1, 2): 1, (3, 2): 0, (0, 0): 1, (3, 3): 4, (3, 0): 0, (3, 1): 0, (1, 1): 2, (2, 1): 0, (0, 2): 1, (2, 0): 0, (1, 3): 0, (2, 3): 1, (2, 2): 3, (1, 0): 0, (0, 3): 0})
    >>> is_invertible(M)
    True

    >>> M1 = Mat(({0,1,2},{0,1,2}),{(0,0):1,(0,2):2,(1,2):3,(2,2):4})
    >>> is_invertible(M1)
    False
    '''
    
    return M.D[0] == M.D[1] and my_is_independent(list(mat2coldict(M).values()))

In [142]:
M = Mat(({0, 1, 2, 3}, {0, 1, 2, 3}), {(0, 1): 0, (1, 2): 1, (3, 2): 0, (0, 0): 1, (3, 3): 4, (3, 0): 0, (3, 1): 0, (1, 1): 2, (2, 1): 0, (0, 2): 1, (2, 0): 0, (1, 3): 0, (2, 3): 1, (2, 2): 3, (1, 0): 0, (0, 3): 0})
is_invertible(M)

True

In [143]:
M1 = Mat(({0,1,2},{0,1,2}),{(0,0):1,(0,2):2,(1,2):3,(2,2):4})
is_invertible(M1)

False

In [9]:
## 11: (Problem 6.7.13) Inverse of a Matrix over GF(2)
def find_matrix_inverse(A):
    '''
    Input:
        - A: an invertible Mat over GF(2)
    Output:
        - A Mat that is the inverse of A
    Examples:
        >>> M1 = Mat(({0,1,2}, {0,1,2}), {(0, 1): one, (1, 0): one, (2, 2): one})
        >>> find_matrix_inverse(M1) == Mat(M1.D, {(0, 1): one, (1, 0): one, (2, 2): one})
        True
        >>> M2 = Mat(({0,1,2,3},{0,1,2,3}),{(0,1):one,(1,0):one,(2,2):one})
        >>> find_matrix_inverse(M2) == Mat(M2.D, {(0, 1): one, (1, 0): one, (2, 2): one})
        True
    '''
    R = A.D[0]; C = A.D[1]
    return coldict2mat([solve(A, Vec(C, {i:one})) for i in C])
    

In [10]:
M1 = Mat(({0,1,2}, {0,1,2}), {(0, 1): one, (1, 0): one, (2, 2): one})
find_matrix_inverse(M1) == Mat(M1.D, {(0, 1): one, (1, 0): one, (2, 2): one})

True

In [11]:
M2 = Mat(({0,1,2,3},{0,1,2,3}),{(0,1):one,(1,0):one,(2,2):one})
find_matrix_inverse(M2) == Mat(M2.D, {(0, 1): one, (1, 0): one, (2, 2): one})

True

In [20]:
## 12: (Problem 6.7.14) Inverse of a Triangular Matrix
def find_triangular_matrix_inverse(A):
    '''
    Supporting GF2 is not required.

    Input:
        - A: an upper triangular Mat with nonzero diagonal elements
    Output:
        - Mat that is the inverse of A
    
    Example:
        >>> A = listlist2mat([[1, .5, .2, 4],[0, 1, .3, .9],[0,0,1,.1],[0,0,0,1]])
        >>> find_triangular_matrix_inverse(A) == Mat(({0, 1, 2, 3}, {0, 1, 2, 3}), {(0, 1): -0.5, (1, 2): -0.3, (3, 2): 0.0, (0, 0): 1.0, (3, 3): 1.0, (3, 0): 0.0, (3, 1): 0.0, (2, 1): 0.0, (0, 2): -0.05000000000000002, (2, 0): 0.0, (1, 3): -0.87, (2, 3): -0.1, (2, 2): 1.0, (1, 0): 0.0, (0, 3): -3.545, (1, 1): 1.0})
        True
    '''
    label_list = list(A.D[1])
    rowlist = list(mat2rowdict(A).values())
    return coldict2mat([triangular_solve(rowlist, label_list, Vec(A.D[1], {i:1})) for i in A.D[1]])

In [21]:
A = listlist2mat([[1, .5, .2, 4],[0, 1, .3, .9],[0,0,1,.1],[0,0,0,1]])
find_triangular_matrix_inverse(A) == Mat(({0, 1, 2, 3}, {0, 1, 2, 3}), {(0, 1): -0.5, (1, 2): -0.3, (3, 2): 0.0, (0, 0): 1.0, (3, 3): 1.0, (3, 0): 0.0, (3, 1): 0.0, (2, 1): 0.0, (0, 2): -0.05000000000000002, (2, 0): 0.0, (1, 3): -0.87, (2, 3): -0.1, (2, 2): 1.0, (1, 0): 0.0, (0, 3): -3.545, (1, 1): 1.0})

True